monte carlo for policy II

In [1]:
import numpy as np
import random
from random import sample
import math 
from collections import defaultdict
import itertools 

In [2]:
weibull_scale=(2365.08,996.88,713.55,1406.84,343.76,3933.12,828.19,2040.95)
weibull_shape=(414.16,109.25,79.81,115.21,169.81,143.60,43.83,296.48)
tf=(2,6.5,2.5,6,5,3.5,3,3.5)
tp=(0.4,5.42,0.625,0.857,1.25,0.7,0.429,0.875)
time_interval=5
running_time=100000

In [3]:
def reset1():
    st= [0]*16
    return tuple(st)

In [4]:
def step (action,st,i): 
    
    f = random.weibullvariate(weibull_scale[i],weibull_shape[i])
    if action == 0 :
        st[i] +=5
        if f <= st[i]:
            st[i+8]=1    
        else:
            st[i+8]=0 
            
    if action ==1 :
            st[i]=0
            st[i+8]=0
           
    return tuple(st)

In [5]:
def rewardfun(action,st,i) :
    reward =[]

    if action ==1 :
        reward= -(time_interval / tp[i])*tp[i]    
    if (st[i+8]==1 and action == 0):
        reward= -(time_interval / tp[i])*time_interval * math.ceil(tf[i]/time_interval)
    
    if (st[i+8]== 0 and action == 0):
        reward = 5
    return reward
    

In [6]:
ALL_POSSIBLE_ACTIONS = [0,1]
def policy_using_pi(st, pi):
    return np.random.choice(ALL_POSSIBLE_ACTIONS, p=[pi[st][a] for a in ALL_POSSIBLE_ACTIONS])

In [7]:
#policy_using_pi((1,1), pi)

In [8]:
def choose_action(state , pi,i):
    if (state[i+8]==1):
        return  1  #repleace with probability 1
    else: 
        st = (state[i],state[i+8])
        return policy_using_pi(st, pi)  #epsilon_soft 

In [9]:
state = (1,0,2,5,1,4,5,0,0,0,0,0,0,0,0,0)

In [10]:
st = (state[1],state[1+8])
st

(0, 0)

In [11]:
#choose_action(state , pi ,1)

For Tire

In [28]:
# initialize 
Q = np.zeros((100000 ,2) + (2,))
returns =np.zeros((100000 ,2) + (2,))
N = np.zeros((100000 ,2) + (2,))
pi = np.full((100000 ,2) + (2,),0.5)
#pi = defaultdict(lambda: 1/len(ALL_POSSIBLE_ACTIONS))# probability of action

i=0
for epi in range(1000 +1):# Looping through episodes 
    
    # Initializes the state
    s_ti = reset1()   
    epsilon = 1/(epi+1)
    G = 0 # Store cumulative reward in G (initialized at 0)
            
    for j in range(running_time//time_interval):
        
        a_t = choose_action(s_ti , pi ,1)
        s_ti = step(a_t,list(s_ti),i)
        G = rewardfun(a_t,s_ti,i)
        
        seen_state_action_pairs = set()
        
        s_t = (s_ti[i] , s_ti[i+8] )
        state_action = (s_t,a_t)
        
        if state_action not in seen_state_action_pairs: #first_visit
            
            returns[s_t][a_t] += G
            N[s_t][a_t] +=1
                
            Q[s_t][a_t] = returns[s_t][a_t] /N[s_t][a_t] # Average reward across episodes
            
            seen_state_action_pairs.add(state_action)
                
                 
            A_star = np.argmax(Q[s_t])# Finding the action with maximum value
                
                
            for a in ALL_POSSIBLE_ACTIONS: # Update action probability for s_t in policy
                if a == A_star:
                    pi[s_t][a] = 1 - epsilon + (epsilon / len(ALL_POSSIBLE_ACTIONS))
                else:
                    pi[s_t][a] = (epsilon / len(ALL_POSSIBLE_ACTIONS))


In [52]:
i=0
time_replace = []
current_state = reset1()
for j in range(running_time//time_interval):
    
    action = np.argmax(Q[(current_state[i],current_state[i+8])])
    if action ==1:
        time_replace.append(current_state[i])
    obs = step(action , list(current_state),i)
    reward = rewardfun(action,current_state,i)
    current_state = obs

np.unique(time_replace)[1] 

2350

Transmission

In [53]:
# initialize 
Q = np.zeros((100000 ,2) + (2,))
returns =np.zeros((100000 ,2) + (2,))
N = np.zeros((100000 ,2) + (2,))
pi = np.full((100000 ,2) + (2,),0.5)
#pi = defaultdict(lambda: 1/len(ALL_POSSIBLE_ACTIONS))# probability of action

i=1
for epi in range(1000 +1):# Looping through episodes 
    
    # Initializes the state
    s_ti = reset1()   
    epsilon = 1/(epi+1)
    G = 0 # Store cumulative reward in G (initialized at 0)
            
    for j in range(running_time//time_interval):
        
        a_t = choose_action(s_ti , pi ,1)
        s_ti = step(a_t,list(s_ti),i)
        G = rewardfun(a_t,s_ti,i)
        
        seen_state_action_pairs = set()
        
        s_t = (s_ti[i] , s_ti[i+8] )
        state_action = (s_t,a_t)
        
        if state_action not in seen_state_action_pairs: #first_visit
            
            returns[s_t][a_t] += G
            N[s_t][a_t] +=1
                
            Q[s_t][a_t] = returns[s_t][a_t] /N[s_t][a_t] # Average reward across episodes
            
            seen_state_action_pairs.add(state_action)
                
                 
            A_star = np.argmax(Q[s_t])# Finding the action with maximum value
                
                
            for a in ALL_POSSIBLE_ACTIONS: # Update action probability for s_t in policy
                if a == A_star:
                    pi[s_t][a] = 1 - epsilon + (epsilon / len(ALL_POSSIBLE_ACTIONS))
                else:
                    pi[s_t][a] = (epsilon / len(ALL_POSSIBLE_ACTIONS))


In [73]:
i=1
time_replace = []
current_state = reset1()
for j in range(running_time//time_interval):
    action = np.argmax(Q[(current_state[i],current_state[i+8])])
    if action ==1:
        time_replace.append(current_state[i])
    obs = step(action , list(current_state),i)
    reward = rewardfun(action,current_state,i)
    current_state = obs

np.unique(time_replace)    

array([ 955,  960,  965,  970,  975,  980,  985,  990,  995, 1000, 1005])

In [74]:
np.unique(time_replace)[6]

985

Wheel

In [75]:
# initialize 
Q = np.zeros((100000 ,2) + (2,))
returns =np.zeros((100000 ,2) + (2,))
N = np.zeros((100000 ,2) + (2,))
pi = np.full((100000 ,2) + (2,),0.5)
#pi = defaultdict(lambda: 1/len(ALL_POSSIBLE_ACTIONS))# probability of action

i=2
for epi in range(1000 +1):# Looping through episodes 
    
    # Initializes the state
    s_ti = reset1()   
    epsilon = 1/(epi+1)
    G = 0 # Store cumulative reward in G (initialized at 0)
            
    for j in range(running_time//time_interval):
        
        a_t = choose_action(s_ti , pi ,1)
        s_ti = step(a_t,list(s_ti),i)
        G = rewardfun(a_t,s_ti,i)
        
        seen_state_action_pairs = set()
        
        s_t = (s_ti[i] , s_ti[i+8] )
        state_action = (s_t,a_t)
        
        if state_action not in seen_state_action_pairs: #first_visit
            
            returns[s_t][a_t] += G
            N[s_t][a_t] +=1
                
            Q[s_t][a_t] = returns[s_t][a_t] /N[s_t][a_t] # Average reward across episodes
            
            seen_state_action_pairs.add(state_action)
                
                 
            A_star = np.argmax(Q[s_t])# Finding the action with maximum value
                
                
            for a in ALL_POSSIBLE_ACTIONS: # Update action probability for s_t in policy
                if a == A_star:
                    pi[s_t][a] = 1 - epsilon + (epsilon / len(ALL_POSSIBLE_ACTIONS))
                else:
                    pi[s_t][a] = (epsilon / len(ALL_POSSIBLE_ACTIONS))

In [76]:
i=2
time_replace = []
current_state = reset1()
for j in range(running_time//time_interval):
    
    action = np.argmax(Q[(current_state[i],current_state[i+8])])
    if action ==1:
        time_replace.append(current_state[i])
    obs = step(action , list(current_state),i)
    reward = rewardfun(action,current_state,i)
    current_state = obs

np.unique(time_replace) 

array([655, 675, 680, 685, 690, 695, 700, 705, 710, 715, 720, 725])

In [78]:
np.unique(time_replace)[4] 

690

Coupling

In [79]:
# initialize 
Q = np.zeros((100000 ,2) + (2,))
returns =np.zeros((100000 ,2) + (2,))
N = np.zeros((100000 ,2) + (2,))
pi = np.full((100000 ,2) + (2,),0.5)
#pi = defaultdict(lambda: 1/len(ALL_POSSIBLE_ACTIONS))# probability of action

i=3
for epi in range(1000 +1):# Looping through episodes 
    
    # Initializes the state
    s_ti = reset1()   
    epsilon = 1/(epi+1)
    G = 0 # Store cumulative reward in G (initialized at 0)
            
    for j in range(running_time//time_interval):
        
        a_t = choose_action(s_ti , pi ,1)
        s_ti = step(a_t,list(s_ti),i)
        G = rewardfun(a_t,s_ti,i)
        
        seen_state_action_pairs = set()
        
        s_t = (s_ti[i] , s_ti[i+8] )
        state_action = (s_t,a_t)
        
        if state_action not in seen_state_action_pairs: #first_visit
            
            returns[s_t][a_t] += G
            N[s_t][a_t] +=1
                
            Q[s_t][a_t] = returns[s_t][a_t] /N[s_t][a_t] # Average reward across episodes
            
            seen_state_action_pairs.add(state_action)
                
                 
            A_star = np.argmax(Q[s_t])# Finding the action with maximum value
                
                
            for a in ALL_POSSIBLE_ACTIONS: # Update action probability for s_t in policy
                if a == A_star:
                    pi[s_t][a] = 1 - epsilon + (epsilon / len(ALL_POSSIBLE_ACTIONS))
                else:
                    pi[s_t][a] = (epsilon / len(ALL_POSSIBLE_ACTIONS))

In [81]:
i=3
time_replace = []
current_state = reset1()
for j in range(running_time//time_interval):
    
    action = np.argmax(Q[(current_state[i],current_state[i+8])])
    if action ==1:
        time_replace.append(current_state[i])
    obs = step(action , list(current_state),i)
    reward = rewardfun(action,current_state,i)
    current_state = obs

np.unique(time_replace)

array([1325, 1355, 1360, 1365, 1370, 1375, 1380, 1385, 1390, 1395, 1400,
       1405, 1410, 1415])

In [82]:
np.unique(time_replace)[2]

1360

Motor

In [84]:
# initialize 
Q = np.zeros((100000 ,2) + (2,))
returns =np.zeros((100000 ,2) + (2,))
N = np.zeros((100000 ,2) + (2,))
pi = np.full((100000 ,2) + (2,),0.5)
#pi = defaultdict(lambda: 1/len(ALL_POSSIBLE_ACTIONS))# probability of action

i=4
for epi in range(1000 +1):# Looping through episodes 
    
    # Initializes the state
    s_ti = reset1()   
    epsilon = 1/(epi+1)
    G = 0 # Store cumulative reward in G (initialized at 0)
            
    for j in range(running_time//time_interval):
        
        a_t = choose_action(s_ti , pi ,1)
        s_ti = step(a_t,list(s_ti),i)
        G = rewardfun(a_t,s_ti,i)
        
        seen_state_action_pairs = set()
        
        s_t = (s_ti[i] , s_ti[i+8] )
        state_action = (s_t,a_t)
        
        if state_action not in seen_state_action_pairs: #first_visit
            
            returns[s_t][a_t] += G
            N[s_t][a_t] +=1
                
            Q[s_t][a_t] = returns[s_t][a_t] /N[s_t][a_t] # Average reward across episodes
            
            seen_state_action_pairs.add(state_action)
                
                 
            A_star = np.argmax(Q[s_t])# Finding the action with maximum value
                
                
            for a in ALL_POSSIBLE_ACTIONS: # Update action probability for s_t in policy
                if a == A_star:
                    pi[s_t][a] = 1 - epsilon + (epsilon / len(ALL_POSSIBLE_ACTIONS))
                else:
                    pi[s_t][a] = (epsilon / len(ALL_POSSIBLE_ACTIONS))

In [85]:
i=4
time_replace = []
current_state = reset1()
for j in range(running_time//time_interval):
    
    action = np.argmax(Q[(current_state[i],current_state[i+8])])
    if action ==1:
        time_replace.append(current_state[i])
    obs = step(action , list(current_state),i)
    reward = rewardfun(action,current_state,i)
    current_state = obs

np.unique(time_replace)

array([335, 340, 345, 350])

In [86]:
np.unique(time_replace)[0]

335

Brake

In [12]:
# initialize 
Q = np.zeros((100000 ,2) + (2,))
returns =np.zeros((100000 ,2) + (2,))
N = np.zeros((100000 ,2) + (2,))
pi = np.full((100000 ,2) + (2,),0.5)
#pi = defaultdict(lambda: 1/len(ALL_POSSIBLE_ACTIONS))# probability of action

i=5
for epi in range(1000 +1):# Looping through episodes 
    
    # Initializes the state
    s_ti = reset1()   
    epsilon = 1/(epi+1)
    G = 0 # Store cumulative reward in G (initialized at 0)
            
    for j in range(running_time//time_interval):
        
        a_t = choose_action(s_ti , pi ,1)
        s_ti = step(a_t,list(s_ti),i)
        G = rewardfun(a_t,s_ti,i)
        
        seen_state_action_pairs = set()
        
        s_t = (s_ti[i] , s_ti[i+8] )
        state_action = (s_t,a_t)
        
        if state_action not in seen_state_action_pairs: #first_visit
            
            returns[s_t][a_t] += G
            N[s_t][a_t] +=1
                
            Q[s_t][a_t] = returns[s_t][a_t] /N[s_t][a_t] # Average reward across episodes
            
            seen_state_action_pairs.add(state_action)
                
                 
            A_star = np.argmax(Q[s_t])# Finding the action with maximum value
                
                
            for a in ALL_POSSIBLE_ACTIONS: # Update action probability for s_t in policy
                if a == A_star:
                    pi[s_t][a] = 1 - epsilon + (epsilon / len(ALL_POSSIBLE_ACTIONS))
                else:
                    pi[s_t][a] = (epsilon / len(ALL_POSSIBLE_ACTIONS))

In [21]:
i=5
time_replace = []
current_state = reset1()
for j in range(running_time//time_interval):
    
    action = np.argmax(Q[(current_state[i],current_state[i+8])])
    if action ==1:
        time_replace.append(current_state[i])
    obs = step(action , list(current_state),i)
    reward = rewardfun(action,current_state,i)
    current_state = obs

np.unique(time_replace)

array([3790, 3830, 3835, 3840, 3845, 3855, 3865, 3870, 3875, 3880, 3885,
       3890, 3895, 3900, 3915, 3925])

In [22]:
np.unique(time_replace)[1]

3830

Steering Wheel

In [23]:
# initialize 
Q = np.zeros((100000 ,2) + (2,))
returns =np.zeros((100000 ,2) + (2,))
N = np.zeros((100000 ,2) + (2,))
pi = np.full((100000 ,2) + (2,),0.5)
#pi = defaultdict(lambda: 1/len(ALL_POSSIBLE_ACTIONS))# probability of action

i=6
for epi in range(1000 +1):# Looping through episodes 
    
    # Initializes the state
    s_ti = reset1()   
    epsilon = 1/(epi+1)
    G = 0 # Store cumulative reward in G (initialized at 0)
            
    for j in range(running_time//time_interval):
        
        a_t = choose_action(s_ti , pi ,1)
        s_ti = step(a_t,list(s_ti),i)
        G = rewardfun(a_t,s_ti,i)
        
        seen_state_action_pairs = set()
        
        s_t = (s_ti[i] , s_ti[i+8] )
        state_action = (s_t,a_t)
        
        if state_action not in seen_state_action_pairs: #first_visit
            
            returns[s_t][a_t] += G
            N[s_t][a_t] +=1
                
            Q[s_t][a_t] = returns[s_t][a_t] /N[s_t][a_t] # Average reward across episodes
            
            seen_state_action_pairs.add(state_action)
                
                 
            A_star = np.argmax(Q[s_t])# Finding the action with maximum value
                
                
            for a in ALL_POSSIBLE_ACTIONS: # Update action probability for s_t in policy
                if a == A_star:
                    pi[s_t][a] = 1 - epsilon + (epsilon / len(ALL_POSSIBLE_ACTIONS))
                else:
                    pi[s_t][a] = (epsilon / len(ALL_POSSIBLE_ACTIONS))

In [24]:
i=6
time_replace = []
current_state = reset1()
for j in range(running_time//time_interval):
    
    action = np.argmax(Q[(current_state[i],current_state[i+8])])
    if action ==1:
        time_replace.append(current_state[i])
    obs = step(action , list(current_state),i)
    reward = rewardfun(action,current_state,i)
    current_state = obs

np.unique(time_replace)

array([735, 740, 745, 750, 755, 760, 765, 770, 775, 780, 785, 790, 795,
       800, 805, 810, 815, 820, 825])

In [25]:
np.unique(time_replace)[5]

760

Shifting Gears

In [26]:
# initialize 
Q = np.zeros((100000 ,2) + (2,))
returns =np.zeros((100000 ,2) + (2,))
N = np.zeros((100000 ,2) + (2,))
pi = np.full((100000 ,2) + (2,),0.5)
#pi = defaultdict(lambda: 1/len(ALL_POSSIBLE_ACTIONS))# probability of action

i=7
for epi in range(1000 +1):# Looping through episodes 
    
    # Initializes the state
    s_ti = reset1()   
    epsilon = 1/(epi+1)
    G = 0 # Store cumulative reward in G (initialized at 0)
            
    for j in range(running_time//time_interval):
        
        a_t = choose_action(s_ti , pi ,1)
        s_ti = step(a_t,list(s_ti),i)
        G = rewardfun(a_t,s_ti,i)
        
        seen_state_action_pairs = set()
        
        s_t = (s_ti[i] , s_ti[i+8] )
        state_action = (s_t,a_t)
        
        if state_action not in seen_state_action_pairs: #first_visit
            
            returns[s_t][a_t] += G
            N[s_t][a_t] +=1
                
            Q[s_t][a_t] = returns[s_t][a_t] /N[s_t][a_t] # Average reward across episodes
            
            seen_state_action_pairs.add(state_action)
                
                 
            A_star = np.argmax(Q[s_t])# Finding the action with maximum value
                
                
            for a in ALL_POSSIBLE_ACTIONS: # Update action probability for s_t in policy
                if a == A_star:
                    pi[s_t][a] = 1 - epsilon + (epsilon / len(ALL_POSSIBLE_ACTIONS))
                else:
                    pi[s_t][a] = (epsilon / len(ALL_POSSIBLE_ACTIONS))

In [33]:
i=7
time_replace = []
current_state = reset1()
for j in range(running_time//time_interval):
    
    action = np.argmax(Q[(current_state[i],current_state[i+8])])
    if action ==1:
        time_replace.append(current_state[i])
    obs = step(action , list(current_state),i)
    reward = rewardfun(action,current_state,i)
    current_state = obs

np.unique(time_replace)

array([1995, 2005, 2010, 2025, 2030, 2035, 2040, 2045, 2050])

In [34]:
np.unique(time_replace)[0]

1995

In [35]:
# initialize 
Q = np.zeros((100000 ,2) + (2,))
returns =np.zeros((100000 ,2) + (2,))
N = np.zeros((100000 ,2) + (2,))
pi = np.full((100000 ,2) + (2,),0.5)
#pi = defaultdict(lambda: 1/len(ALL_POSSIBLE_ACTIONS))# probability of action

i=7
for epi in range(1500 +1):# Looping through episodes 
    
    # Initializes the state
    s_ti = reset1()   
    epsilon = 1/(epi+1)
    G = 0 # Store cumulative reward in G (initialized at 0)
            
    for j in range(running_time//time_interval):
        
        a_t = choose_action(s_ti , pi ,1)
        s_ti = step(a_t,list(s_ti),i)
        G = rewardfun(a_t,s_ti,i)
        
        seen_state_action_pairs = set()
        
        s_t = (s_ti[i] , s_ti[i+8] )
        state_action = (s_t,a_t)
        
        if state_action not in seen_state_action_pairs: #first_visit
            
            returns[s_t][a_t] += G
            N[s_t][a_t] +=1
                
            Q[s_t][a_t] = returns[s_t][a_t] /N[s_t][a_t] # Average reward across episodes
            
            seen_state_action_pairs.add(state_action)
                
                 
            A_star = np.argmax(Q[s_t])# Finding the action with maximum value
                
                
            for a in ALL_POSSIBLE_ACTIONS: # Update action probability for s_t in policy
                if a == A_star:
                    pi[s_t][a] = 1 - epsilon + (epsilon / len(ALL_POSSIBLE_ACTIONS))
                else:
                    pi[s_t][a] = (epsilon / len(ALL_POSSIBLE_ACTIONS))

In [48]:
i=7
time_replace = []
current_state = reset1()
for j in range(running_time//time_interval):
    
    action = np.argmax(Q[(current_state[i],current_state[i+8])])
    if action ==1:
        time_replace.append(current_state[i])
    obs = step(action , list(current_state),i)
    reward = rewardfun(action,current_state,i)
    current_state = obs

np.unique(time_replace)

array([2000, 2010, 2020, 2025, 2030, 2035, 2040, 2045, 2050])